In [2]:
import pandas as pd
import numpy as np
import time

path = 'nba_data_processed.csv'
df = pd.read_csv(filepath_or_buffer=path, sep= ',', header=0)
df.dropna(how='all',inplace=True)
df.fillna(0,inplace=True)
print(df.shape)
qsna=df.shape[0]-df.isnull().sum(axis=0)
qna=df.isnull().sum(axis=0)
ppna=round(100*(df.isnull().sum(axis=0)/df.shape[0]),2)
aux= {'datos sin NAs en q': qsna, 'Na en q': qna ,'Na en %': ppna}
na=pd.DataFrame(data=aux)
na.sort_values(by='Na en %',ascending=False)

(624, 29)


,datos sin NAs en q,Na en q,Na en %
Player,624,0,0.0
Pos,624,0,0.0
Age,624,0,0.0
Tm,624,0,0.0
G,624,0,0.0
GS,624,0,0.0
MP,624,0,0.0
FG,624,0,0.0
FGA,624,0,0.0
FG%,624,0,0.0


In [3]:
df.head()

,Player,Pos,Age,Tm,G,GS,MP,FG,FGA,FG%,...,FT%,ORB,DRB,TRB,AST,STL,BLK,TOV,PF,PTS
0,Precious Achiuwa,C,23.0,TOR,38.0,11.0,23.3,3.9,8.1,0.482,...,0.689,2.0,4.6,6.6,1.0,0.6,0.7,1.2,2.1,10.2
1,Steven Adams,C,29.0,MEM,42.0,42.0,27.0,3.7,6.3,0.597,...,0.364,5.1,6.5,11.5,2.3,0.9,1.1,1.9,2.3,8.6
2,Bam Adebayo,C,25.0,MIA,57.0,57.0,35.0,8.4,15.7,0.536,...,0.800,2.6,7.2,9.8,3.2,1.2,0.8,2.5,2.8,21.2
3,Ochai Agbaji,SG,22.0,UTA,39.0,2.0,15.6,1.8,3.8,0.483,...,0.682,0.7,1.1,1.8,0.6,0.2,0.1,0.3,1.4,5.0
4,Santi Aldama,PF,22.0,MEM,56.0,18.0,22.0,3.3,7.0,0.474,...,0.729,1.0,3.6,4.6,1.2,0.7,0.7,0.7,1.9,9.4


## Conceptos

### El método Group by y métodos de agregación

Agrupar datos implica dividir un conjunto de datos en partes más pequeñas basadas en una o más características comunes. En pandas, para eso usamos GroupBy

El proceso de groupby en Pandas generalmente implica tres etapas:

1. División (Splitting): Dividir el DataFrame en grupos según un criterio dado.
2. Aplicación (Applying): Aplicar una función a cada grupo de manera independiente.
3. Combinación (Combining): Combinar los resultados de las funciones aplicadas en una estructura de datos.

Este proceso se conoce como el «paradigma Split-Apply-Combine».

#### Aplicaciones Comunes de groupby

##### Agregación (Aggregation): La agregación implica aplicar una función que resuma los datos de cada grupo. Las funciones de agregación comunes incluyen sum, mean, count, min, max, etc.

In [4]:
grouped = df.groupby(['Tm', 'Pos'])
grouped.agg({'PTS': 'mean', 'G': 'sum', 'Age': 'max'}).reset_index()

,Tm,Pos,PTS,G,Age
0,ATL,C,6.525000,132.0,29.0
1,ATL,PF,13.200000,51.0,25.0
2,ATL,PG,8.800000,145.0,26.0
3,ATL,SF,9.950000,169.0,25.0
4,ATL,SG,7.850000,145.0,33.0
...,...,...,...,...,...
153,WAS,C,8.800000,158.0,37.0
154,WAS,PF,9.450000,131.0,30.0
155,WAS,PG,5.583333,145.0,30.0
156,WAS,SF,6.633333,112.0,24.0


##### Transformación (Transformation): La transformación implica aplicar una función a cada elemento de un grupo, produciendo un DataFrame del mismo tamaño que el original. Las funciones de transformación comunes incluyen apply, transform, etc

In [5]:
grouped['Player'].transform(lambda x: x.str.split().str[0])

0      Precious
1        Steven
2           Bam
3         Ochai
4         Santi
         ...   
644    McKinley
645    Thaddeus
646        Trae
647        Cody
648       Ivica
Name: Player, Length: 624, dtype: object

##### Filtrado (Filtering): El filtrado implica aplicar una condición a cada grupo y devolver un subconjunto del DataFrame que cumple con esa condición.

In [6]:
grouped.filter(lambda x: x['G'].min()>10)

,Player,Pos,Age,Tm,G,GS,MP,FG,FGA,FG%,...,FT%,ORB,DRB,TRB,AST,STL,BLK,TOV,PF,PTS
1,Steven Adams,C,29.0,MEM,42.0,42.0,27.0,3.7,6.3,0.597,...,0.364,5.1,6.5,11.5,2.3,0.9,1.1,1.9,2.3,8.6
3,Ochai Agbaji,SG,22.0,UTA,39.0,2.0,15.6,1.8,3.8,0.483,...,0.682,0.7,1.1,1.8,0.6,0.2,0.1,0.3,1.4,5.0
6,Nickeil Alexander-Walker,SG,24.0,UTA,36.0,3.0,14.7,2.3,4.7,0.488,...,0.692,0.2,1.4,1.6,2.1,0.7,0.4,1.3,1.6,6.3
8,Grayson Allen,SG,27.0,MIL,56.0,54.0,27.6,3.4,7.8,0.440,...,0.906,0.9,2.5,3.4,2.4,0.8,0.2,1.1,1.6,10.5
9,Jarrett Allen,C,24.0,CLE,57.0,57.0,33.2,6.2,9.4,0.654,...,0.733,3.2,6.7,9.9,1.6,0.8,1.2,1.5,2.2,14.8
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
639,James Wiseman,C,21.0,TOT,25.0,2.0,14.2,3.3,5.3,0.617,...,0.674,1.0,3.2,4.2,0.6,0.1,0.3,0.8,2.0,7.9
640,James Wiseman,C,21.0,GSW,21.0,0.0,12.5,2.8,4.5,0.628,...,0.684,0.9,2.6,3.5,0.7,0.1,0.3,0.7,1.9,6.9
642,Christian Wood,C,27.0,DAL,50.0,17.0,27.4,6.4,12.2,0.525,...,0.763,1.4,6.6,8.0,1.7,0.4,1.2,1.9,2.7,17.6
645,Thaddeus Young,PF,34.0,TOR,49.0,9.0,15.5,2.1,3.8,0.562,...,0.692,1.4,1.8,3.2,1.4,1.1,0.1,0.8,1.8,4.7


### El método Apply


**Sintaxis Básica**

La sintaxis básica del método apply es:

DataFrame.apply(func, axis=0, raw=False, result_type=None, args=(), **kwds)

* func: La función que se va a aplicar.
* axis: El eje a lo largo del cual se aplica la función (0 para columnas, 1 para filas).
* raw: Si es True, la función se aplica a los valores de la matriz, en lugar de a los Series.
* result_type: Puede ser ‘expand’, ‘reduce’, ‘broadcast’, o None.
* args y **kwds: Argumentos y palabras clave adicionales que se pasan a la función.
 

**Importancia del Método apply**

El método apply es fundamental para varias tareas en el análisis de datos debido a su flexibilidad:

* Personalización: Permite aplicar funciones personalizadas que no están disponibles en las funciones de agregación estándar.
* Versatilidad: Puede utilizarse para transformar, filtrar, y agregar datos de maneras muy específicas.
* Compatibilidad: Funciona bien con otras funciones y métodos de Pandas, lo que facilita la integración en pipelines de datos complejos.


Consideraciones y Mejores Prácticas al Usar apply

* Rendimiento: El método apply puede ser más lento que las funciones vectorizadas de Pandas. Siempre que sea posible, intenta usar funciones vectorizadas (.mean(), .sum(), etc.) en lugar de apply.
* Simplicidad: Usa apply para funciones que no se pueden realizar fácilmente con métodos incorporados. No uses apply para operaciones simples que tienen métodos directos en Pandas.
* Comprensibilidad: Mantén las funciones aplicadas a apply lo más simples y claras posible. Las funciones complejas pueden ser difíciles de depurar y mantener.
Uso de lambda: Aunque lambda es útil para funciones pequeñas y simples, considera definir funciones por separado para operaciones más complejas para mejorar la legibilidad.

In [7]:
# Ejemplo Apply: Calcular la cantidad de tiros aproximados por partido que tiró cada jugado
start_time = time.time()
df['FG_total'] = df['FG'] / df['FG%']
execution_time = time.time() - start_time
print(execution_time*1000)

0.0


In [8]:
start_time = time.time()
df['FG_total'] = df.apply(lambda x: x['FG'] / x['FG%'] if x['FG%']>0 else 0, axis=1)
execution_time = time.time() - start_time
print(execution_time*1000)

3.5126209259033203


## **Ejercicios**

In [9]:
# Paso 1: Agrupar los datos por equipo y posición
agrupado_por_equipo_posicion = df.groupby(['Tm', 'Pos']).agg({
    'PTS': 'mean',  # Promedio de puntos
    'AST': 'mean',  # Promedio de asistencias
    'TRB': 'mean'   # Promedio de rebotes
}).reset_index()

print(agrupado_por_equipo_posicion)

      Tm Pos        PTS       AST       TRB
0    ATL   C   6.525000  0.775000  5.250000
1    ATL  PF  13.200000  1.200000  7.000000
2    ATL  PG   8.800000  3.425000  1.600000
3    ATL  SF   9.950000  0.950000  3.675000
4    ATL  SG   7.850000  1.766667  2.416667
..   ...  ..        ...       ...       ...
153  WAS   C   8.800000  1.125000  4.225000
154  WAS  PF   9.450000  1.400000  3.300000
155  WAS  PG   5.583333  2.566667  2.316667
156  WAS  SF   6.633333  1.433333  2.966667
157  WAS  SG  10.533333  2.633333  2.333333

[158 rows x 5 columns]


In [10]:
# Paso 2: Calcular la eficiencia de los jugadores
# La eficiencia se define como la suma de puntos, asistencias y rebotes por juego.
df['eficiencia'] = df['PTS'] + df['AST'] + df['TRB']
# Agrupar por equipo y posición, y calcular la eficiencia promedio por posición en cada equipo
eficiencia_por_equipo_posicion = df.groupby(['Tm', 'Pos']).apply(
    lambda x: pd.Series({
        'Eficiencia Promedio': x['eficiencia'].mean()
    })
).reset_index()
print(eficiencia_por_equipo_posicion)

      Tm Pos  Eficiencia Promedio
0    ATL   C            12.550000
1    ATL  PF            21.400000
2    ATL  PG            13.825000
3    ATL  SF            14.575000
4    ATL  SG            12.033333
..   ...  ..                  ...
153  WAS   C            14.150000
154  WAS  PF            14.150000
155  WAS  PG            10.466667
156  WAS  SF            11.033333
157  WAS  SG            15.500000

[158 rows x 3 columns]


C:\Users\Diego\AppData\Local\Temp\ipykernel_27276\1470240936.py:5: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  eficiencia_por_equipo_posicion = df.groupby(['Tm', 'Pos']).apply(


In [11]:
# Paso 3: Identificar al jugador más eficiente por equipo y posición
jugador_mas_eficiente = df.groupby(['Tm', 'Pos']).apply(
    lambda x: x.loc[x['eficiencia'].idxmax(), ['Player', 'eficiencia']]
).reset_index()

print(jugador_mas_eficiente)


      Tm Pos              Player  eficiencia
0    ATL   C        Clint Capela        24.0
1    ATL  PF        John Collins        21.4
2    ATL  PG          Trae Young        40.1
3    ATL  SF     De'Andre Hunter        21.3
4    ATL  SG     Dejounte Murray        32.5
..   ...  ..                 ...         ...
153  WAS   C  Kristaps Porziņģis        33.9
154  WAS  PF          Kyle Kuzma        32.6
155  WAS  PG        Monte Morris        19.1
156  WAS  SF         Deni Avdija        17.9
157  WAS  SG        Bradley Beal        31.6

[158 rows x 4 columns]


C:\Users\Diego\AppData\Local\Temp\ipykernel_27276\1742863198.py:2: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  jugador_mas_eficiente = df.groupby(['Tm', 'Pos']).apply(


In [14]:

mas_eficiente = df.loc[df.groupby('Tm')['eficiencia'].idxmax(), ['Tm', 'Player', 'eficiencia']]
print(mas_eficiente)

      Tm                   Player  eficiencia
646  ATL               Trae Young        40.1
563  BOS             Jayson Tatum        43.5
159  BRK             Kevin Durant        41.7
136  CHI            DeMar DeRozan        35.0
25   CHO              LaMelo Ball        38.1
407  CLE         Donovan Mitchell        36.0
146  DAL              Luka Dončić        50.0
301  DEN             Nikola Jokić        46.5
125  DET          Cade Cunningham        32.1
127  GSW            Stephen Curry        42.1
473  HOU         Kevin Porter Jr.        30.3
222  IND        Tyrese Haliburton        33.7
190  LAC              Paul George        34.8
289  LAL             LeBron James        44.8
414  MEM                Ja Morant        41.0
2    MIA              Bam Adebayo        34.2
12   MIL    Giannis Antetokounmpo        48.7
162  MIN          Anthony Edwards        35.1
637  NOP          Zion Williamson        37.6
489  NYK            Julius Randle        39.7
193  OKC  Shai Gilgeous-Alexander 